In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as D
import torchvision.utils as vutils
from torchvision import datasets
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torch.utils.data as data

from torchsummary import summary

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import torchvision.models as models
import os
import copy
import time
import csv

from PIL import Image

from tensorboardX import SummaryWriter

import matplotlib.pyplot as plt

In [2]:
# Training settings
batch_size = 64

In [3]:
# Training settings

t_image_size=224
transform_train = transforms.Compose([
    transforms.Resize(t_image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])
transform_val = transforms.Compose([
    transforms.Resize(t_image_size),
    transforms.ToTensor()
])
transform_test = transforms.Compose([
    transforms.Resize(t_image_size),
    transforms.ToTensor()
])

data_dir = '/media/lee/E61C94F21C94BECD/place_dataset/Places2/data_256/5'

train_image_datasets = torchvision.datasets.ImageFolder(os.path.join(data_dir),
                                     transform = transform_train)
val_image_datasets = torchvision.datasets.ImageFolder(os.path.join(data_dir),
                                     transform = transform_val)
test_image_datasets = torchvision.datasets.ImageFolder(os.path.join(data_dir),
                                     transform = transform_test)


dataset_size = len(train_image_datasets)
print('total size:', dataset_size)
class_names_train = train_image_datasets.classes
class_names_val = val_image_datasets.classes
class_names_test = test_image_datasets.classes
# print(class_names)
print('# of train class: ', len(class_names_train))
print('# of valid class: ', len(class_names_val))
print('# of test class : ', len(class_names_test))
## SPLIT DATASET
train_split= 0.7
validate_split = 0.20
test_split = 0.10
train_size = int(train_split * dataset_size)
validation_size = int(validate_split * dataset_size)
test_size = int(dataset_size - train_size - validation_size)

# ########### CURRENTLY DOING THIS, WHICH WORKS ###########

indices = list(range(dataset_size))
np.random.shuffle(indices)
train_indices = indices[:train_size]
temp = int(train_size+validation_size)
val_indices = indices[train_size:temp]
test_indices = indices[temp:]
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(
    train_image_datasets, batch_size=batch_size, sampler=train_sampler,
    num_workers=4)
valid_loader = torch.utils.data.DataLoader(
    val_image_datasets, batch_size=batch_size, sampler=valid_sampler,
    num_workers=4)
test_loader = torch.utils.data.DataLoader(
    test_image_datasets, batch_size=batch_size, sampler=test_sampler,
    num_workers=4)

dataloaders = {
    'train': train_loader,
    'valid': valid_loader,
    'test' : test_loader
}
image_datasets = {
    'train': train_sampler,
    'valid': valid_sampler,
    'test' : test_sampler
}

dataset_sizes ={
    'train': len(image_datasets['train']),
    'valid': len(image_datasets['valid']),
    'test' : len(image_datasets['test'])
}

# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# print('train_class:' len(train_image_datasets.classes))

print('train_size: ',dataset_sizes['train'], '\nvalid_size: ',dataset_sizes['valid'], 
      '\ntest_size : ',dataset_sizes['test'])
# class_names = image_datasets['train'].classes
# print(class_names)
# print('len class: ', len(class_names))

total size: 23469
# of train class:  5
# of valid class:  5
# of test class :  5
train_size:  16428 
valid_size:  4693 
test_size :  2348


In [4]:
from pytorch_mobilenet.benchmark import MobileNet

In [5]:
model = MobileNet()
model.fc = nn.Linear(1024,5)
# MobileNet.fc = nn.Linear(1024,5)
print(model)

MobileNet(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias

In [6]:
# model = MobileNet()
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA, model")
    model.cuda() #after second other epoch model
summary(model,(3,224,224))

Using CUDA, model
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              ReLU-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 64, 112, 112]           2,048
       BatchNorm2d-8         [-1, 64, 112, 112]             128
              ReLU-9         [-1, 64, 112, 112]               0
           Conv2d-10           [-1, 64, 56, 56]             576
      BatchNorm2d-11           [-1, 64, 56, 56]             128
             ReLU-12           [-1, 64, 56, 56]               0
           Conv2d-13          [-1, 128, 56, 56]           8,192
      BatchNorm2d-14 

In [7]:
writer = SummaryWriter()

In [8]:
def train_model(model, scheduler, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)
        since1 = time.time()
        # Each epoch has a training and validation phase
        train_batches = len(dataloaders['train'])
        for phase in ['train','valid']:
            
            print("lr:", optimizer.param_groups[0]['lr'])
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            for i, data in enumerate(dataloaders[phase]):
                if i % 100 == 0:
                    print("\rTraining batch {}/{}".format(i, len(dataloaders[phase])), end='', flush=True)
                # Use half training dataset
                if i >= len(dataloaders[phase]):
                    break    
                inputs, labels = data
                inputs, labels = Variable(inputs.cuda()),Variable(labels.cuda())

                optimizer.zero_grad()

                    # forward
                    # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    out = model(inputs)
                    _, preds = torch.max(out, 1)
                    loss = criterion(out, labels)
#                     print(loss)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()


                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            if phase == 'train':
                writer.add_scalar('data/train_loss_places', epoch_loss, epoch)
                writer.add_scalar('data/train_acc_places', epoch_acc, epoch)
            else:
                writer.add_scalar('data/val_loss_places', epoch_loss, epoch)
                writer.add_scalar('data/val_acc_places', epoch_acc, epoch)
            for name, param in model.named_parameters():
                writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)
            time_elapsed1 = time.time() - since1
            print('\rEpoch process in {:.0f}m {:.0f}s'.format(
            time_elapsed1 // 60, time_elapsed1 % 60))
            print('{} Loss: {:.4f} Acc: {:.4f} lr: {:.4f}'.format(phase, epoch_loss, 
                                                                  epoch_acc, optimizer.param_groups[0]['lr']))
#             csvfile = open(os.path.join('./csv/resnet18/', '20190102resnet18_64_data_15_places_{}{}_class{}_epoch{}.csv'.format(optimizer_name, learning_rate, len(class_names), num_epochs)), 'a', newline='')
#             csv_writer = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
#             csv_writer.writerow(['class', len(class_names), 'epoch', epoch, phase, epoch_loss, epoch_acc])
#             csvfile.close()
#             deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f} at {}'.format(best_acc, best_epoch+1))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [9]:
import copy
lr = 0.001  # 0.1
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=0.0001)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)
total_epoch = 10
model_train = train_model(model, scheduler, criterion, optimizer, num_epochs=total_epoch)
torch.save(model_train.state_dict(), './Trained/place_50_adam.pt')

Epoch 1/10
----------
lr: 0.001
Epoch process in 2m 50s
train Loss: 1.0168 Acc: 0.5791 lr: 0.0010
lr: 0.001
Epoch process in 3m 34s
valid Loss: 0.9553 Acc: 0.6167 lr: 0.0010

Epoch 2/10
----------
lr: 0.001
Epoch process in 0m 41s
train Loss: 0.7989 Acc: 0.6853 lr: 0.0010
lr: 0.001
Epoch process in 0m 48s
valid Loss: 0.7439 Acc: 0.7047 lr: 0.0010

Epoch 3/10
----------
lr: 0.001
Epoch process in 0m 38s
train Loss: 0.6963 Acc: 0.7233 lr: 0.0010
lr: 0.001
Epoch process in 0m 45s
valid Loss: 0.7666 Acc: 0.6925 lr: 0.0010

Epoch 4/10
----------
lr: 0.001
Epoch process in 0m 39s
train Loss: 0.6456 Acc: 0.7440 lr: 0.0010
lr: 0.001
Epoch process in 0m 46s
valid Loss: 0.6984 Acc: 0.7172 lr: 0.0010

Epoch 5/10
----------
lr: 0.001
Epoch process in 0m 40s
train Loss: 0.6101 Acc: 0.7577 lr: 0.0010
lr: 0.001
Epoch process in 0m 49s
valid Loss: 0.6847 Acc: 0.7309 lr: 0.0010

Epoch 6/10
----------
lr: 0.001
Epoch process in 0m 39s
train Loss: 0.5817 Acc: 0.7669 lr: 0.0010
lr: 0.001
Epoch process in 